<h1>Automated Gating with Immunova</h1>

This is the first of three experiments comparing the automated gating in immunova to expert manual gating. For this experiment I will be using pre-gated data from peritoneal dialysis patients.

<h3>Create project and experiments</h3>

In [2]:
from immunova.data.project import Project
from immunova.data.mongo_setup import global_init
from immunova.data.fcs_experiments import FCSExperiment, Panel
from immunova.data.utilities import get_fcs_file_paths
from tqdm import tqdm_notebook, tqdm
import pandas as pd
global_init()

In [3]:
pd_project = Project(project_id='Peritonitis', owner='burtonrossj')
pd_project.save()

<Project: Project object>

In [4]:
pd_project.id.__str__()

'5d811990c05e87e82a538365'

I will create four experiments in total:
* PBMC_T: T cell panel for PBMC samples
* PBMC_M: Myeloid cell panel for PBMC samples
* PDMC_T: T cell panel for peritoneal fluid samples
* PDMC_M: Myeloid cell panel for peritoneal fluid samples

For each of these experiments I need to associate a flow cytometry panel. A Panel object defines the channel(fluorochrome)/marker(antibody) mappings for all associated flow data. This allows for standardisation of the flow cytometry meta-data at the point of entry.

Panel objects can be created from a python dictionary object or using an excel template. In this case I have created an excel template (see documentation for details on creating panel templates)

In [5]:
t_panel = Panel(panel_name='peritonitis_t_panel')
m_panel = Panel(panel_name='peritonitis_m_panel')

In [6]:
t_panel.create_from_excel(path='experiment_data/peritonitis_t_template.xlsx')

True

In [7]:
m_panel.create_from_excel(path='experiment_data/peritonitis_m_template.xlsx')

True

The `create_from_excel` method will populate the Panel object using the excel template. I can now save the panels to the database.

In [8]:
t_panel.save()
m_panel.save()

<Panel: Panel object>

With the panels created I can now create the experiments. When you create an experiment you must always associate it to a project. We therefore use the `add_experiment` method of the Project object.

In [9]:
pbmc_t = pd_project.add_experiment(experiment_id='PBMC_T', panel_name='peritonitis_t_panel')
pdmc_t = pd_project.add_experiment(experiment_id='PDMC_T', panel_name='peritonitis_t_panel')
pbmc_m = pd_project.add_experiment(experiment_id='PBMC_M', panel_name='peritonitis_m_panel')
pdmc_m = pd_project.add_experiment(experiment_id='PDMC_M', panel_name='peritonitis_m_panel')

Experiment created successfully!
Experiment created successfully!
Experiment created successfully!
Experiment created successfully!


Now that the experiments are created I can start adding the fcs files. The `add_new_sample` method is used to generate a new fcs file entry into the mongo database, which is then associated to the experiment. See the documentation below:


In [10]:
?pbmc_t.add_new_sample

Signature:
pbmc_t.add_new_sample(
    sample_id:str,
    file_path:str,
    controls:list,
    comp_matrix:<built-in function array>=None,
    compensate:bool=True,
    feedback:bool=True,
) -> str
Docstring:
Add a new sample (FileGroup) to this experiment
:param sample_id: primary ID for identification of sample (FileGroup.primary_id)
:param file_path: file path of the primary fcs file (e.g. the fcs file that is of primary interest such as the
file with complete staining)
:param controls: list of file paths for control files e.g. a list of file paths for associated FMO controls
:param comp_matrix: (optional) numpy array for spillover matrix for compensation calculation; if not supplied
the matrix linked within the fcs file will be used, if not present will present an error
:param compensate: boolean value as to whether compensation should be applied before data entry (default=True)
:param feedback: boolean value, if True function will provide feedback in the form of print statements
(

<h3>Add files</h3>

A summary table of all the samples collected in the peritonitis study can provide us with the sample numbers and the manual gating results.

In [11]:
summary = pd.read_excel('/media/ross/FCS_DATA/Raya PD Samples/ClinicalData_and_ManualGatingResults.xlsx')

In [12]:
pdmc_sample_ids = summary[summary['Cell origin'] == 'PDMC']['Patient no.'].values

In [13]:
pdmc_sample_ids

array(['142-09', '175-09', '209-03', '209-05', '210-12', '210-14',
       '229-02', '237-06', '239-02', '239-04', '251-07', '251-08',
       '254-04', '254-05', '255-04', '255-05', '262-01', '264-02',
       '267-01', '267-02', '272-01', '273-01', '276-01', '279-03',
       '286-02', '286-03', '286-04', '288-02', '289-01', '294-01',
       '294-02', '294-03', '295-01', '298-01', '302-01', '305-01',
       '305-02', '305-03', '306-01', '307-01', '308-01', '308-02R',
       '308-03R', '308-04', '310-01', '315-01', '315-02', '316-01',
       '318-01', '320-01', '321-01', '322-01', '323-01', '323-02',
       '324-01', '326-01'], dtype=object)

In [14]:
get_fcs_file_paths(fcs_dir='/media/ross/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel',
                  control_names=['CD1c', 'HLA-DR'], ctrl_id='FMO')

{'primary': ['/media/ross/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel/Peri 142-09R PDMC 1 N Panel_N1_013.fcs'],
 'controls': [{'control_id': 'CD1c',
   'path': '/media/ross/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel/Peri 142-09R PDMC 1 N Panel_N2 FMO CD1c_014.fcs'},
  {'control_id': 'HLA-DR',
   'path': '/media/ross/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel/Peri 142-09R PDMC 1 N Panel_N2 FMO CD1c_014.fcs'},
  {'control_id': 'CD1c',
   'path': '/media/ross/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel/Peri 142-09R PDMC 1 N Panel_N3 FMO HLA-DR_015.fcs'},
  {'control_id': 'HLA-DR',
   'path': '/media/ross/FCS_DATA/Raya PD Samples/ds_friendly/PDMC/142-09/m_panel/Peri 142-09R PDMC 1 N Panel_N3 FMO HLA-DR_015.fcs'}]}